In [47]:
import itk
import nibabel as nib
import numpy as np
import nrrd

ImagePath1 = "Z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-42-Spiromics-67503688/Markups/88-Adam/S-Final-2-label.nrrd"
ImagePath2 = "Z:/E-Kelly-VidaSegmentals&Sublobes/3- 20Spiromics-10ForKelly-ForERS23/Case42-IA210257_H-17900_20190805_125316_20230524_121115_export/ZUNU_vida-airtree-recentered.nii.gz"
#Image 2 is the airway while image 1 is the arterial tree. We will take image 2 and resample it using image 1 geometry
#Image 1 is an nrrd and Image 2 is a nii.gz

image1 = nib.load(ImagePath1)
im1 = image1 = image1.get_fdata()

image2 = nib.load(ImagePath2)
im2 = image2.get_fdata()

#Getting image geometries (Origin, orientation(fixed), spacing, and size)

#Image 1
origin1 = image1.affine[:3, 3]
orientation1 = image1.affine[:3, :3]
spacing1 = image1.header.get_zooms()
size1 = im1.shape

#Image 2
origin2 = image2.affine[:3, 3]
orientation2 = image2.affine[:3, :3]
spacing2 = image2.header.get_zooms()
size2 = im2.shape


ImageFileError: Cannot work out file type of "Z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-42-Spiromics-67503688/Markups/88-Adam/S-Final-2-label.nrrd"

In [46]:
image2.affine

array([[  -0.68359375,    0.        ,    0.        ,  174.65820312],
       [   0.        ,   -0.68359375,    0.        ,  358.15820312],
       [   0.        ,    0.        ,    0.5       , -338.1000061 ],
       [   0.        ,    0.        ,    0.        ,    1.        ]])